In [13]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, write_submission, create_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores,MergeTwoModelsByScores

##Best config is 159: {'topK': 656, 'alpha': 0.5902046712405923}

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13917 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13908 ( 0.5%) Users that have less than 1 test interactions


In [4]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [5]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

In [6]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_train)
SLIME_recommender.fit(topK=417, l1_ratio=0.010668404033843646, alpha=0.0022461592936665505, workers = 20)

100%|████████████████████████████████████▉| 18000/18059 [08:58<00:00, 60.88it/s]

In [7]:
similarity_rp3 = RP3Beta_recommender.W_sparse
similarity_slime = SLIME_recommender.W_sparse

In [9]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real,Integer

hyperparameters_range_dictionary = {
    "topK": Integer(200,700),
    "alpha": Real(0.0, 1.0)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_rp3, similarity_slime],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 347, 'alpha': 0.05078495975101928}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 9.23 sec. Users per second: 1508
SearchBayesianSkopt: New best config found. Config 0: {'topK': 347, 'alpha': 0.05078495975101928} - results: PRECISION: 0.1982539, PRECISION_RECALL_MIN_DEN: 0.2025493, RECALL: 0.0581587, MAP: 0.0932118, MAP_MIN_DEN: 0.0953224, MRR: 0.4200929, NDCG: 0.2085232, F1: 0.0899347, HIT_RATE: 0.8639075, ARHR_ALL_HITS: 0.6238377, NOVELTY: 0.0061894, AVERAGE_POPULARITY: 0.2847178, DIVERSITY_MEAN_INTER_LIST: 0.9808170, DIVERSITY_HERFINDAHL: 0.9980747, COVERAGE_ITEM: 0.2167894, COVERAGE_ITEM_CORRECT: 0.1230411, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8597683, DIVERSITY_GINI: 0.0421890, SHANNON_ENTROPY: 9.8953280, RATIO_DIVERSITY_HERFINDAHL: 0.9984429, RATIO_DIVERSITY_GINI: 0.1579245, 

EvaluatorHoldout: Processed 13917 (100.0%) in 9.25 sec. Users per second: 1504
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'topK': 379, 'alpha': 0.14291072437741917} - results: PRECISION: 0.2007832, PRECISION_RECALL_MIN_DEN: 0.2050274, RECALL: 0.0588566, MAP: 0.0945147, MAP_MIN_DEN: 0.0965857, MRR: 0.4213034, NDCG: 0.2106809, F1: 0.0910293, HIT_RATE: 0.8668535, ARHR_ALL_HITS: 0.6291888, NOVELTY: 0.0061475, AVERAGE_POPULARITY: 0.2997380, DIVERSITY_MEAN_INTER_LIST: 0.9797286, DIVERSITY_HERFINDAHL: 0.9979658, COVERAGE_ITEM: 0.2133008, COVERAGE_ITEM_CORRECT: 0.1202171, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8627002, DIVERSITY_GINI: 0.0401343, SHANNON_ENTROPY: 9.8193093, RATIO_DIVERSITY_HERFINDAHL: 0.9983340, RATIO_DIVERSITY_GINI: 0.1502330, RATIO_SHANNON_ENTROPY: 0.7868002, RATIO_AVERAGE_POPULARITY: 1.5321305, RATIO_NOVELTY: 0.0324650, 

Iteration No: 5 ended. Evaluation done at random point.
Time taken: 11.1379
Function value obtained: -0.0945
Current minimum: -0.100

EvaluatorHoldout: Processed 13917 (100.0%) in 10.00 sec. Users per second: 1391
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'topK': 483, 'alpha': 0.5583579705473988} - results: PRECISION: 0.2086728, PRECISION_RECALL_MIN_DEN: 0.2131005, RECALL: 0.0615000, MAP: 0.0984431, MAP_MIN_DEN: 0.1005598, MRR: 0.4279955, NDCG: 0.2178045, F1: 0.0950013, HIT_RATE: 0.8770568, ARHR_ALL_HITS: 0.6476331, NOVELTY: 0.0060587, AVERAGE_POPULARITY: 0.3563509, DIVERSITY_MEAN_INTER_LIST: 0.9634676, DIVERSITY_HERFINDAHL: 0.9963398, COVERAGE_ITEM: 0.2035550, COVERAGE_ITEM_CORRECT: 0.1086439, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8728547, DIVERSITY_GINI: 0.0325415, SHANNON_ENTROPY: 9.3764983, RATIO_DIVERSITY_HERFINDAHL: 0.9967074, RATIO_DIVERSITY_GINI: 0.1218112, RATIO_SHANNON_ENTROPY: 0.7513187, RATIO_AVERAGE_POPULARITY: 1.8215113, RATIO_NOVELTY: 0.0319962, 

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 12.1455
Function value obtained: -0.0984
Current minimum: -0.1

EvaluatorHoldout: Processed 13917 (100.0%) in 9.63 sec. Users per second: 1445
SearchBayesianSkopt: Config 17 is suboptimal. Config: {'topK': 413, 'alpha': 0.8869551639003544} - results: PRECISION: 0.2071136, PRECISION_RECALL_MIN_DEN: 0.2111969, RECALL: 0.0599566, MAP: 0.0976883, MAP_MIN_DEN: 0.0996498, MRR: 0.4268549, NDCG: 0.2161207, F1: 0.0929930, HIT_RATE: 0.8707336, ARHR_ALL_HITS: 0.6435339, NOVELTY: 0.0057437, AVERAGE_POPULARITY: 0.5074125, DIVERSITY_MEAN_INTER_LIST: 0.8738129, DIVERSITY_HERFINDAHL: 0.9873750, COVERAGE_ITEM: 0.1754804, COVERAGE_ITEM_CORRECT: 0.0770253, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8665618, DIVERSITY_GINI: 0.0149359, SHANNON_ENTROPY: 7.8064234, RATIO_DIVERSITY_HERFINDAHL: 0.9877393, RATIO_DIVERSITY_GINI: 0.0559091, RATIO_SHANNON_ENTROPY: 0.6255120, RATIO_AVERAGE_POPULARITY: 2.5936730, RATIO_NOVELTY: 0.0303323, 

Iteration No: 18 ended. Evaluation done at random point.
Time taken: 11.6187
Function value obtained: -0.0977
Current minimum: -0.10

EvaluatorHoldout: Processed 13917 (100.0%) in 10.14 sec. Users per second: 1373
SearchBayesianSkopt: Config 24 is suboptimal. Config: {'topK': 567, 'alpha': 0.25806128986211246} - results: PRECISION: 0.2077890, PRECISION_RECALL_MIN_DEN: 0.2121737, RECALL: 0.0609639, MAP: 0.0979609, MAP_MIN_DEN: 0.1001018, MRR: 0.4271962, NDCG: 0.2168664, F1: 0.0942697, HIT_RATE: 0.8753323, ARHR_ALL_HITS: 0.6447434, NOVELTY: 0.0060549, AVERAGE_POPULARITY: 0.3416875, DIVERSITY_MEAN_INTER_LIST: 0.9716095, DIVERSITY_HERFINDAHL: 0.9971540, COVERAGE_ITEM: 0.2029459, COVERAGE_ITEM_CORRECT: 0.1118002, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8711384, DIVERSITY_GINI: 0.0345472, SHANNON_ENTROPY: 9.5376092, RATIO_DIVERSITY_HERFINDAHL: 0.9975219, RATIO_DIVERSITY_GINI: 0.1293189, RATIO_SHANNON_ENTROPY: 0.7642282, RATIO_AVERAGE_POPULARITY: 1.7465584, RATIO_NOVELTY: 0.0319762, 

Iteration No: 25 ended. Evaluation done at random point.
Time taken: 12.2123
Function value obtained: -0.0980
Current minimum: -0.

...

Iteration No: 166 ended. Search finished for the next optimal point.
Time taken: 13.7042
Function value obtained: -0.1010
Current minimum: -0.1010
Iteration No: 167 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 370, 'alpha': 0.8464327696339348}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 9.29 sec. Users per second: 1499
SearchBayesianSkopt: Config 166 is suboptimal. Config: {'topK': 370, 'alpha': 0.8464327696339348} - results: PRECISION: 0.2081483, PRECISION_RECALL_MIN_DEN: 0.2122847, RECALL: 0.0604751, MAP: 0.0984345, MAP_MIN_DEN: 0.1004868, MRR: 0.4290917, NDCG: 0.2174315, F1: 0.0937208, HIT_RATE: 0.8723144, ARHR_ALL_HITS: 0.6476918, NOVELTY: 0.0057630, AVERAGE_POPULARITY: 0.4976923, DIVERSITY_MEAN_INTER_LIST: 0.8814716, DIVERSITY_HERFINDAHL: 0.9881408, COVERAGE_ITEM: 0.1774185, COVERAGE_ITEM_CORRECT: 0.0800709, COVERAGE_USER: 0.9952088, COVERAGE_USE

Iteration No: 173 ended. Search finished for the next optimal point.
Time taken: 13.9165
Function value obtained: -0.1010
Current minimum: -0.1010
Iteration No: 174 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 261, 'alpha': 0.6818133414232472}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 8.18 sec. Users per second: 1701
SearchBayesianSkopt: Config 173 is suboptimal. Config: {'topK': 261, 'alpha': 0.6818133414232472} - results: PRECISION: 0.1970324, PRECISION_RECALL_MIN_DEN: 0.2014668, RECALL: 0.0584507, MAP: 0.0919671, MAP_MIN_DEN: 0.0939920, MRR: 0.4144865, NDCG: 0.2064936, F1: 0.0901561, HIT_RATE: 0.8609614, ARHR_ALL_HITS: 0.6157440, NOVELTY: 0.0062783, AVERAGE_POPULARITY: 0.2843568, DIVERSITY_MEAN_INTER_LIST: 0.9792823, DIVERSITY_HERFINDAHL: 0.9979212, COVERAGE_ITEM: 0.2313528, COVERAGE_ITEM_CORRECT: 0.1246470, COVERAGE_USER: 0.9952088, COVERAGE_USE

Iteration No: 180 ended. Search finished for the next optimal point.
Time taken: 13.9514
Function value obtained: -0.1010
Current minimum: -0.1010
Iteration No: 181 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 660, 'alpha': 0.5900030340396496}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 9.90 sec. Users per second: 1406
SearchBayesianSkopt: Config 180 is suboptimal. Config: {'topK': 660, 'alpha': 0.5900030340396496} - results: PRECISION: 0.2132572, PRECISION_RECALL_MIN_DEN: 0.2177864, RECALL: 0.0628468, MAP: 0.1010247, MAP_MIN_DEN: 0.1032486, MRR: 0.4347984, NDCG: 0.2224453, F1: 0.0970832, HIT_RATE: 0.8811525, ARHR_ALL_HITS: 0.6609490, NOVELTY: 0.0058907, AVERAGE_POPULARITY: 0.4292233, DIVERSITY_MEAN_INTER_LIST: 0.9314873, DIVERSITY_HERFINDAHL: 0.9931420, COVERAGE_ITEM: 0.1918711, COVERAGE_ITEM_CORRECT: 0.0966831, COVERAGE_USER: 0.9952088, COVERAGE_USE

Iteration No: 187 ended. Search finished for the next optimal point.
Time taken: 14.2614
Function value obtained: -0.1010
Current minimum: -0.1010
Iteration No: 188 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 660, 'alpha': 0.5900636342663453}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 9.86 sec. Users per second: 1411
SearchBayesianSkopt: Config 187 is suboptimal. Config: {'topK': 660, 'alpha': 0.5900636342663453} - results: PRECISION: 0.2132644, PRECISION_RECALL_MIN_DEN: 0.2177936, RECALL: 0.0628440, MAP: 0.1010273, MAP_MIN_DEN: 0.1032511, MRR: 0.4348439, NDCG: 0.2224543, F1: 0.0970806, HIT_RATE: 0.8812244, ARHR_ALL_HITS: 0.6609859, NOVELTY: 0.0058907, AVERAGE_POPULARITY: 0.4292452, DIVERSITY_MEAN_INTER_LIST: 0.9314745, DIVERSITY_HERFINDAHL: 0.9931408, COVERAGE_ITEM: 0.1917050, COVERAGE_ITEM_CORRECT: 0.0966277, COVERAGE_USER: 0.9952088, COVERAGE_USE

Iteration No: 194 ended. Search finished for the next optimal point.
Time taken: 14.3715
Function value obtained: -0.1010
Current minimum: -0.1010
Iteration No: 195 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 464, 'alpha': 0.4743761092742399}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 9.73 sec. Users per second: 1431
SearchBayesianSkopt: Config 194 is suboptimal. Config: {'topK': 464, 'alpha': 0.4743761092742399} - results: PRECISION: 0.2067759, PRECISION_RECALL_MIN_DEN: 0.2112169, RECALL: 0.0609760, MAP: 0.0974870, MAP_MIN_DEN: 0.0996012, MRR: 0.4261240, NDCG: 0.2160500, F1: 0.0941795, HIT_RATE: 0.8738952, ARHR_ALL_HITS: 0.6428524, NOVELTY: 0.0061009, AVERAGE_POPULARITY: 0.3331897, DIVERSITY_MEAN_INTER_LIST: 0.9722923, DIVERSITY_HERFINDAHL: 0.9972222, COVERAGE_ITEM: 0.2101445, COVERAGE_ITEM_CORRECT: 0.1150119, COVERAGE_USER: 0.9952088, COVERAGE_USE

In [8]:

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_train,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=656, alpha=0.5902046712405923 )

In [9]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_train)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

100%|████████████████████████████████████▉| 18056/18059 [09:10<00:00, 60.88it/s]

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 34.11 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 1.16 min
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 1.33 min
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 1.51 min
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 1.68 min
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 1.85 min
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 2.01 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 2.17 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 2.35 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 2.51 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 2.51 min


In [14]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0)
}
recommenders = [similarity_hybrid, IALS_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )



##Best config is 135: {'alpha': 0.8828405982112033, 'beta': 0.41993188895272643}

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.5350140122715806, 'beta': 0.26715900248181296}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 12.35 sec. Users per second: 1127
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.5350140122715806, 'beta': 0.26715900248181296} - results: PRECISION: 0.2174535, PRECISION_RECALL_MIN_DEN: 0.2218130, RECALL: 0.0636838, MAP: 0.1035233, MAP_MIN_DEN: 0.1056571, MRR: 0.4402185, NDCG: 0.2264746, F1: 0.0985160, HIT_RATE: 0.8907811, ARHR_ALL_HITS: 0.6727893, NOVELTY: 0.0056962, AVERAGE_POPULARITY: 0.4821276, DIVERSITY_MEAN_INTER_LIST: 0.9285598, DIVERSITY_HERFINDAHL: 0.9928493, COVERAGE_ITEM: 0.1376599, COVERAGE_ITEM_CORRECT: 0.0709342, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8865132, DIVERSITY_GINI: 0.0159610, SHANNON_ENTROPY: 8.3248134, RATIO_DIVERSITY_HERFINDAHL: 0.9932156, RATIO_DIVERSITY

EvaluatorHoldout: Processed 13917 (100.0%) in 12.22 sec. Users per second: 1139
SearchBayesianSkopt: Config 6 is suboptimal. Config: {'alpha': 0.3239924133882636, 'beta': 0.016688894832501692} - results: PRECISION: 0.2148380, PRECISION_RECALL_MIN_DEN: 0.2193918, RECALL: 0.0635269, MAP: 0.1018148, MAP_MIN_DEN: 0.1040570, MRR: 0.4363853, NDCG: 0.2239249, F1: 0.0980583, HIT_RATE: 0.8848171, ARHR_ALL_HITS: 0.6647888, NOVELTY: 0.0058471, AVERAGE_POPULARITY: 0.4415441, DIVERSITY_MEAN_INTER_LIST: 0.9293401, DIVERSITY_HERFINDAHL: 0.9929273, COVERAGE_ITEM: 0.1797995, COVERAGE_ITEM_CORRECT: 0.0913118, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8805778, DIVERSITY_GINI: 0.0220736, SHANNON_ENTROPY: 8.6382842, RATIO_DIVERSITY_HERFINDAHL: 0.9932937, RATIO_DIVERSITY_GINI: 0.0826271, RATIO_SHANNON_ENTROPY: 0.6921672, RATIO_AVERAGE_POPULARITY: 2.2569820, RATIO_NOVELTY: 0.0308787, 

Iteration No: 7 ended. Evaluation done at random point.
Time taken: 12.3355
Function value obtained: -0.1018
Curren

EvaluatorHoldout: Processed 13917 (100.0%) in 12.12 sec. Users per second: 1148
SearchBayesianSkopt: Config 12 is suboptimal. Config: {'alpha': 0.23942755240027486, 'beta': 0.06710385752614624} - results: PRECISION: 0.2176116, PRECISION_RECALL_MIN_DEN: 0.2220665, RECALL: 0.0640529, MAP: 0.1033846, MAP_MIN_DEN: 0.1055158, MRR: 0.4397084, NDCG: 0.2264651, F1: 0.0989735, HIT_RATE: 0.8895595, ARHR_ALL_HITS: 0.6721848, NOVELTY: 0.0057414, AVERAGE_POPULARITY: 0.4703234, DIVERSITY_MEAN_INTER_LIST: 0.9274156, DIVERSITY_HERFINDAHL: 0.9927349, COVERAGE_ITEM: 0.1502852, COVERAGE_ITEM_CORRECT: 0.0773576, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8852975, DIVERSITY_GINI: 0.0174838, SHANNON_ENTROPY: 8.4027789, RATIO_DIVERSITY_HERFINDAHL: 0.9931012, RATIO_DIVERSITY_GINI: 0.0654464, RATIO_SHANNON_ENTROPY: 0.6732967, RATIO_AVERAGE_POPULARITY: 2.4040894, RATIO_NOVELTY: 0.0303203, 

Iteration No: 13 ended. Evaluation done at random point.
Time taken: 12.2352
Function value obtained: -0.1034
Curr

EvaluatorHoldout: Processed 13917 (100.0%) in 12.12 sec. Users per second: 1149
SearchBayesianSkopt: Config 19 is suboptimal. Config: {'alpha': 0.9885430505453069, 'beta': 0.207420393602524} - results: PRECISION: 0.2172738, PRECISION_RECALL_MIN_DEN: 0.2217062, RECALL: 0.0639530, MAP: 0.1032294, MAP_MIN_DEN: 0.1054068, MRR: 0.4391218, NDCG: 0.2261615, F1: 0.0988192, HIT_RATE: 0.8880506, ARHR_ALL_HITS: 0.6711732, NOVELTY: 0.0057639, AVERAGE_POPULARITY: 0.4644930, DIVERSITY_MEAN_INTER_LIST: 0.9271461, DIVERSITY_HERFINDAHL: 0.9927079, COVERAGE_ITEM: 0.1568747, COVERAGE_ITEM_CORRECT: 0.0812891, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8837958, DIVERSITY_GINI: 0.0183764, SHANNON_ENTROPY: 8.4479779, RATIO_DIVERSITY_HERFINDAHL: 0.9930742, RATIO_DIVERSITY_GINI: 0.0687876, RATIO_SHANNON_ENTROPY: 0.6769184, RATIO_AVERAGE_POPULARITY: 2.3742867, RATIO_NOVELTY: 0.0304390, 

Iteration No: 20 ended. Evaluation done at random point.
Time taken: 12.2320
Function value obtained: -0.1032
Current

EvaluatorHoldout: Processed 13917 (100.0%) in 12.10 sec. Users per second: 1150
SearchBayesianSkopt: Config 26 is suboptimal. Config: {'alpha': 0.1478131755414667, 'beta': 0.22772094128816472} - results: PRECISION: 0.2140260, PRECISION_RECALL_MIN_DEN: 0.2180776, RECALL: 0.0619445, MAP: 0.1023189, MAP_MIN_DEN: 0.1042835, MRR: 0.4394062, NDCG: 0.2238366, F1: 0.0960808, HIT_RATE: 0.8833800, ARHR_ALL_HITS: 0.6683565, NOVELTY: 0.0056306, AVERAGE_POPULARITY: 0.4965560, DIVERSITY_MEAN_INTER_LIST: 0.9337146, DIVERSITY_HERFINDAHL: 0.9933648, COVERAGE_ITEM: 0.1083116, COVERAGE_ITEM_CORRECT: 0.0588626, COVERAGE_USER: 0.9952088, COVERAGE_USER_CORRECT: 0.8791476, DIVERSITY_GINI: 0.0140180, SHANNON_ENTROPY: 8.2442359, RATIO_DIVERSITY_HERFINDAHL: 0.9937313, RATIO_DIVERSITY_GINI: 0.0524731, RATIO_SHANNON_ENTROPY: 0.6605930, RATIO_AVERAGE_POPULARITY: 2.5381790, RATIO_NOVELTY: 0.0297355, 

Iteration No: 27 ended. Evaluation done at random point.
Time taken: 12.2185
Function value obtained: -0.1023
Curre

...

Iteration No: 169 ended. Search finished for the next optimal point.
Time taken: 15.1959
Function value obtained: -0.1035
Current minimum: -0.1036
Iteration No: 170 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.3717633756281517, 'beta': 0.9868875330295424}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 12.64 sec. Users per second: 1101
SearchBayesianSkopt: Config 169 is suboptimal. Config: {'alpha': 0.3717633756281517, 'beta': 0.9868875330295424} - results: PRECISION: 0.2116907, PRECISION_RECALL_MIN_DEN: 0.2154112, RECALL: 0.0606276, MAP: 0.1013401, MAP_MIN_DEN: 0.1031408, MRR: 0.4370068, NDCG: 0.2216537, F1: 0.0942596, HIT_RATE: 0.8802903, ARHR_ALL_HITS: 0.6634006, NOVELTY: 0.0056124, AVERAGE_POPULARITY: 0.4996664, DIVERSITY_MEAN_INTER_LIST: 0.9357318, DIVERSITY_HERFINDAHL: 0.9935665, COVERAGE_ITEM: 0.0943020, COVERAGE_ITEM_CORRECT: 0.0536575, COVERAGE_USER: 0.995

Iteration No: 176 ended. Search finished for the next optimal point.
Time taken: 15.6929
Function value obtained: -0.1035
Current minimum: -0.1036
Iteration No: 177 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.37273866877658196, 'beta': 0.853063080575404}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 13.40 sec. Users per second: 1039
SearchBayesianSkopt: Config 176 is suboptimal. Config: {'alpha': 0.37273866877658196, 'beta': 0.853063080575404} - results: PRECISION: 0.2122871, PRECISION_RECALL_MIN_DEN: 0.2162411, RECALL: 0.0611401, MAP: 0.1015580, MAP_MIN_DEN: 0.1034500, MRR: 0.4378324, NDCG: 0.2222710, F1: 0.0949375, HIT_RATE: 0.8817274, ARHR_ALL_HITS: 0.6646704, NOVELTY: 0.0056167, AVERAGE_POPULARITY: 0.4989438, DIVERSITY_MEAN_INTER_LIST: 0.9352792, DIVERSITY_HERFINDAHL: 0.9935212, COVERAGE_ITEM: 0.0978459, COVERAGE_ITEM_CORRECT: 0.0545988, COVERAGE_USER: 0.995

Iteration No: 183 ended. Search finished for the next optimal point.
Time taken: 15.4643
Function value obtained: -0.1015
Current minimum: -0.1036
Iteration No: 184 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.45275203077320625, 'beta': 0.0010921817209992304}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 12.54 sec. Users per second: 1110
SearchBayesianSkopt: Config 183 is suboptimal. Config: {'alpha': 0.45275203077320625, 'beta': 0.0010921817209992304} - results: PRECISION: 0.2132859, PRECISION_RECALL_MIN_DEN: 0.2178318, RECALL: 0.0628944, MAP: 0.1010228, MAP_MIN_DEN: 0.1032656, MRR: 0.4348660, NDCG: 0.2224906, F1: 0.0971430, HIT_RATE: 0.8818711, ARHR_ALL_HITS: 0.6609903, NOVELTY: 0.0058880, AVERAGE_POPULARITY: 0.4300588, DIVERSITY_MEAN_INTER_LIST: 0.9312450, DIVERSITY_HERFINDAHL: 0.9931178, COVERAGE_ITEM: 0.1907082, COVERAGE_ITEM_CORRECT: 0.0964616, COVERAGE_USE

Iteration No: 190 ended. Search finished for the next optimal point.
Time taken: 15.7363
Function value obtained: -0.1035
Current minimum: -0.1036
Iteration No: 191 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.28971609289114325, 'beta': 0.48902111577874574}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 12.71 sec. Users per second: 1095
SearchBayesianSkopt: Config 190 is suboptimal. Config: {'alpha': 0.28971609289114325, 'beta': 0.48902111577874574} - results: PRECISION: 0.2133362, PRECISION_RECALL_MIN_DEN: 0.2172867, RECALL: 0.0615609, MAP: 0.1019861, MAP_MIN_DEN: 0.1039187, MRR: 0.4388822, NDCG: 0.2232177, F1: 0.0955497, HIT_RATE: 0.8833082, ARHR_ALL_HITS: 0.6668819, NOVELTY: 0.0056271, AVERAGE_POPULARITY: 0.4971955, DIVERSITY_MEAN_INTER_LIST: 0.9340942, DIVERSITY_HERFINDAHL: 0.9934027, COVERAGE_ITEM: 0.1062628, COVERAGE_ITEM_CORRECT: 0.0578659, COVERAGE_USER: 0

Iteration No: 197 ended. Search finished for the next optimal point.
Time taken: 15.9176
Function value obtained: -0.1027
Current minimum: -0.1036
Iteration No: 198 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.3633659054739214, 'beta': 0.6489220203447275}
MergeTwoModelsByScore: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13917 (100.0%) in 12.52 sec. Users per second: 1111
SearchBayesianSkopt: Config 197 is suboptimal. Config: {'alpha': 0.3633659054739214, 'beta': 0.6489220203447275} - results: PRECISION: 0.2132787, PRECISION_RECALL_MIN_DEN: 0.2171903, RECALL: 0.0614545, MAP: 0.1019359, MAP_MIN_DEN: 0.1038506, MRR: 0.4385241, NDCG: 0.2231058, F1: 0.0954157, HIT_RATE: 0.8830926, ARHR_ALL_HITS: 0.6665670, NOVELTY: 0.0056251, AVERAGE_POPULARITY: 0.4975471, DIVERSITY_MEAN_INTER_LIST: 0.9343242, DIVERSITY_HERFINDAHL: 0.9934257, COVERAGE_ITEM: 0.1047123, COVERAGE_ITEM_CORRECT: 0.0574783, COVERAGE_USER: 0.995

In [11]:
recommenders = [similarity_hybrid, IALS_recommender]
hybrid = MergeTwoModelsByScores(URM_train =URM_train, recommenders=recommenders)
hybrid.fit(alpha=0.8828405982112033, beta=0.41993188895272643)

In [14]:
submission = create_submission(hybrid)
write_submission(submission,"RP3hSLIME_IALS_urm")

100%|████████████████████████████████████▉| 18056/18059 [13:11<00:00, 22.81it/s]
